In [1]:
import time
import pandas as pd
from dask import dataframe as dd
!pip install modin
import modin.pandas as mpd
from distributed import Client
client = Client()
!pip install "ray[data]"
import ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 956 kB 4.8 MB/s 
     |████████████████████████████████| 761 kB 48.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.4 MB 1.2 MB/s 
     |████████████████████████████████| 8.8 MB 52.3 MB/s 
     |████████████████████████████████| 4.1 MB 47.6 MB/s 
     |████████████████████████████████| 468 kB 67.8 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.49.1
    Uninstalling grpcio-1.49.1:
      Successfully uninstalled grpcio-1.49.1


#Write testutility.py file

In [2]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


#Write the YAML file

In [3]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: price_paid_records
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - Transaction_unique_identifier
    - Price
    - Date_of_Transfer
    - Property_Type
    - Old_New  
    - Duration 
    - Town_City
    - District
    - County
    - PPDCategory_Type
    - Record_Status_monthly_file_only

Writing file.yaml


In [4]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [5]:
config_data['inbound_delimiter']

','

In [6]:
#inspecting data of config file
config_data


{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'price_paid_records',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['Transaction_unique_identifier',
  'Price',
  'Date_of_Transfer',
  'Property_Type',
  'Old_New',
  'Duration',
  'Town_City',
  'District',
  'County',
  'PPDCategory_Type',
  'Record_Status_monthly_file_only']}

In [7]:

# read the file using config file and Pandas
start = time.time()
file_type = config_data['file_type']
source_file = "/content/drive/MyDrive/Test_Ingestion/" + config_data['file_name'] + f'.{file_type}'
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
end = time.time()
print("Computational time in reading the file using Pandas:",end-start)

df.head()

Computational time in reading the file using Pandas: 61.910961627960205


,Transaction unique identifier,Price,Date of Transfer,Property Type,Old/New,Duration,Town/City,District,County,PPDCategory Type,Record Status - monthly file only
0,{81B82214-7FBC-4129-9F6B-4956B4A663AD},25000,1995-08-18 00:00,T,N,F,OLDHAM,OLDHAM,GREATER MANCHESTER,A,A
1,{8046EC72-1466-42D6-A753-4956BF7CD8A2},42500,1995-08-09 00:00,S,N,F,GRAYS,THURROCK,THURROCK,A,A
2,{278D581A-5BF3-4FCE-AF62-4956D87691E6},45000,1995-06-30 00:00,T,N,F,HIGHBRIDGE,SEDGEMOOR,SOMERSET,A,A
3,{1D861C06-A416-4865-973C-4956DB12CD12},43150,1995-11-24 00:00,T,N,F,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE,A,A
4,{DD8645FD-A815-43A6-A7BA-4956E58F1874},18899,1995-06-23 00:00,S,N,F,WAKEFIELD,LEEDS,WEST YORKSHIRE,A,A


In [8]:
# read the file using config file and Dask

start = time.time()
file_type = config_data['file_type']
source_file = "/content/drive/MyDrive/Test_Ingestion/" + config_data['file_name'] + f'.{file_type}'
dask_df = dd.read_csv(source_file)
end = time.time()
print("Computational time in reading the file using Pandas:",end-start)

dask_df.head()

Computational time in reading the file using Pandas: 0.2768895626068115


,Transaction unique identifier,Price,Date of Transfer,Property Type,Old/New,Duration,Town/City,District,County,PPDCategory Type,Record Status - monthly file only
0,{81B82214-7FBC-4129-9F6B-4956B4A663AD},25000,1995-08-18 00:00,T,N,F,OLDHAM,OLDHAM,GREATER MANCHESTER,A,A
1,{8046EC72-1466-42D6-A753-4956BF7CD8A2},42500,1995-08-09 00:00,S,N,F,GRAYS,THURROCK,THURROCK,A,A
2,{278D581A-5BF3-4FCE-AF62-4956D87691E6},45000,1995-06-30 00:00,T,N,F,HIGHBRIDGE,SEDGEMOOR,SOMERSET,A,A
3,{1D861C06-A416-4865-973C-4956DB12CD12},43150,1995-11-24 00:00,T,N,F,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE,A,A
4,{DD8645FD-A815-43A6-A7BA-4956E58F1874},18899,1995-06-23 00:00,S,N,F,WAKEFIELD,LEEDS,WEST YORKSHIRE,A,A


Dask performed the task in lesser time compared to Pandas.

In [9]:
#Removing special character , white spaces from the col name
import re

def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace('[^\w]','_',regex=True)
df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
df.columns

Index(['transaction_unique_identifier', 'price', 'date_of_transfer',
       'property_type', 'old_new', 'duration', 'town_city', 'district',
       'county', 'ppdcategory_type', 'record_status_monthly_file_only'],
      dtype='object')

In [10]:
df.columns

Index(['transaction_unique_identifier', 'price', 'date_of_transfer',
       'property_type', 'old_new', 'duration', 'town_city', 'district',
       'county', 'ppdcategory_type', 'record_status_monthly_file_only'],
      dtype='object')

In [11]:
#validate the header of the file

util.col_header_val(df,config_data)

column name and column length validation passed


1

In [12]:
from distributed import config
#Comparing the column names of YAML file and Input File
print("columns of the input dataset are:" ,df.columns)
print("columns of YAML file are:" ,config_data['columns'])


columns of the input dataset are: Index(['transaction_unique_identifier', 'price', 'date_of_transfer',
       'property_type', 'old_new', 'duration', 'town_city', 'district',
       'county', 'ppdcategory_type', 'record_status_monthly_file_only'],
      dtype='object')
columns of YAML file are: ['Transaction_unique_identifier', 'Price', 'Date_of_Transfer', 'Property_Type', 'Old_New', 'Duration', 'Town_City', 'District', 'County', 'PPDCategory_Type', 'Record_Status_monthly_file_only']
[False False False False False False False False False False False]


In [13]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation passed
col validation passed


In [18]:
#comparing the length of the columns in YAML 
print("Length of column of input data file: ",len(df.columns))
print("Length of column of YAML file: ",len(config_data['columns']))
if (len(df.columns)==len(config_data['columns'])):
  print("Length of input data column and length of YAML file columns are equal")
else:
    print("Length of input data column and length of YAML file columns are not equal")


Length of column of input data file:  11
Length of column of YAML file:  11
Length of input data column and length of YAML file columns are equal


In [15]:
gz_file= df.to_csv('gz_compressed.gz',
          sep = '|',
          header = True,
          index = False,
          #quoting = csv.QUOTE_ALL,
          compression = 'gzip',
          quotechar = '"',
          doublequote = True)


In [16]:
gz_file_read=pd.read_csv('/content/gz_compressed.gz', compression='gzip',  sep='|')

In [17]:
gz_file_read.head()

,transaction_unique_identifier,price,date_of_transfer,property_type,old_new,duration,town_city,district,county,ppdcategory_type,record_status_monthly_file_only
0,{81B82214-7FBC-4129-9F6B-4956B4A663AD},25000,1995-08-18 00:00,T,N,F,OLDHAM,OLDHAM,GREATER MANCHESTER,A,A
1,{8046EC72-1466-42D6-A753-4956BF7CD8A2},42500,1995-08-09 00:00,S,N,F,GRAYS,THURROCK,THURROCK,A,A
2,{278D581A-5BF3-4FCE-AF62-4956D87691E6},45000,1995-06-30 00:00,T,N,F,HIGHBRIDGE,SEDGEMOOR,SOMERSET,A,A
3,{1D861C06-A416-4865-973C-4956DB12CD12},43150,1995-11-24 00:00,T,N,F,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE,A,A
4,{DD8645FD-A815-43A6-A7BA-4956E58F1874},18899,1995-06-23 00:00,S,N,F,WAKEFIELD,LEEDS,WEST YORKSHIRE,A,A


The shape of the file is given as below:

In [20]:
df.shape

(22489348, 11)

It is observed that there are 2289348 rows and 11 columns in the dataset.